1\. Write a function that converts numbers among the bin, dec, and hex representations (bin<->dec<->hex)

In [5]:
def representation(number):
    dec, binary, hexadec = False, False, False

    # check which kind of number it is
    if any([(i == "x") for i in number]):
        hexadec = True
    elif any([(i == "b") for i in number]):
        binary = True
    elif (hexadec == False and binary == False):
        dec = True

    if dec == True:
        decimal_representation = int(number)
        binary_representation = bin(decimal_representation)
        hexadecimal_representation = hex(decimal_representation)
    if binary == True:
        binary_representation = number
        decimal_representation = int(number, 2)
        hexadecimal_representation = hex(decimal_representation)
    if hexadec == True:
        hexadecimal_representation = number.replace("#", "")
        decimal_representation = int(hexadecimal_representation, 16)
        binary_representation = bin(decimal_representation)
  

    return (decimal_representation, binary_representation, hexadecimal_representation)

In [6]:
number = str(input("Input number in decimal, binary, or hexadecimal format (for instance: '23', '0b10111' or '0x17'): "))
dec, binary, hexadec = representation(number)

print("\n")
print("Decimal representation: ", dec)
print("Binary representation: ", binary)
print("Hexadecimal representation: ", hexadec)



Decimal representation:  23
Binary representation:  0b10111
Hexadecimal representation:  0x17


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent) and check its validity.

In [7]:
def floating_number(word):
    if len(word) != 32:
        raise ValueError("Not 32-bit word")
    if (i.equals("0") or i.equals("1") for i in word):

        bias = 127
        sign = (-1)**int(word[0])
        exponent = word[1:9]
        mantissa = word[9:]


        f = 0
        for i in range(23):
            f += int(mantissa[i])*2**(-i-1)

        floating_number = sign*(1+f)*2**(int(exponent, 2)-bias)
    else:
        raise ValueError("Not binary")

    return floating_number

In [9]:
# EXAMPLES FROM http://www.dspguide.com/graphics/F_4_2.gif
#example1 = "00000011111000000000000000000000" 1.316554e-36
#example2  = "11000000101100000000000000000000" -5.5

word = input("Input a word: ")
print("\nThe 32-bit word is:", floating_number(word))


The 32-bit word is: -5.5


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tip**: define two variables initialized to 1 and halve/double them for a sufficient amount of times to exceed the under/over-flow limits  

In [10]:
from math import isinf

def limits():
    up = 1.
    down = 1.
    while(isinf(up*2) == False):
        up *= 2
    while(down/2 != 0.):
        down /= 2
    return up, down

print("Over- and underflows (within a factor of 2) are: ", *limits())

Over- and underflows (within a factor of 2) are:  8.98846567431158e+307 5e-324


4\. Write a program to determine the machine precision

**Tip**: define a new variable by adding an increasingly smaller value (in the same way as the previous problem) and check when the addition starts to have no effect on the number

In [11]:
a = 0.
epsilon = 1.

print("\nStarting with a variable a = ", a)
k = 0
while(a+epsilon != a):
    a += epsilon
    epsilon /= 2
    k += 1

print("\nAfter %i reductions of epsilon by half, we found that epsilon has no effect on a." % k)
print("\nThe current value of epsilon is EPS  = %.2e and of a = %f" % (epsilon, a))


Starting with a variable a =  0.0

After 54 reductions of epsilon by half, we found that epsilon has no effect on a.

The current value of epsilon is EPS  = 5.55e-17 and of a = 2.000000


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying the numerator and the denominator by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what has been previously obtained, and why?

(c) write a function that computes the roots of a quadratic equation accurately in all cases

In [12]:
import numpy as np

def quadratic(a, b, c):
    pos_solution = (-b + np.sqrt(b*b-4*a*c))/(2*a)
    neg_solution = (-b - np.sqrt(b*b-4*a*c))/(2*a)
    return pos_solution, neg_solution

sol1, sol2 = quadratic(0.001, 1000, 0.001)
print("The standard equation gives solutions: %e and %e" % (sol1, sol2))

The standard equation gives solutions: -9.999894e-07 and -1.000000e+06


In [13]:
def quadratic_v2(a, b, c):
    pos_solution = 2*c/(-b-np.sqrt(b*b-4*a*c))
    neg_solution = 2*c/(-b+np.sqrt(b*b-4*a*c))
    return pos_solution, neg_solution

sol1_v2, sol2_v2 = quadratic_v2(0.001, 1000, 0.001)
print("The re-expressed equation gives solutions: %e and %e" % (sol1_v2, sol2_v2))

The re-expressed equation gives solutions: -1.000000e-06 and -1.000011e+06


While the first expression yields the correct solution -1e6, the closest to zero solution is wrong by 0.001% approximately.

This is due to the fact that the numerator of the equation is approximately zero (given the fact that we are substracting two numbers very close by in value):

$
-b + \sqrt{b^{2}-4ac} = -1000 + \sqrt{1000^{2}-4\cdot 10 ^{-6}}\approx 0
$

and so the first solution (the one with the $+$ sign) is not "completely" accurate.



The second expression, however, yields solutions as:
$
x_{\pm} = \frac{2c}{-b\mp \sqrt{b^{2}-4ac}}
$

and so the problem arising from the numerator before for the positive solution is arising now from the negative solution (remember the signs plus and minus switched from the first expression to the second). Therefore, now the second solution is off for about 0.001%.

To solve these problems where $b^{2}\gg 4ac$, a combined approach is needed, the negative solution is extracted from the usual formula and the positive solution is extracted from the re-expressed one.

In [14]:
def quadratic_accurate(a, b, c):
    pos_solution = 2*c/(-b-np.sqrt(b*b-4*a*c))
    neg_solution = (-b - np.sqrt(b*b-4*a*c))/(2*a)
    return pos_solution, neg_solution

sol1, sol2 = quadratic_accurate(0.001, 1000, 0.001)
print("The accurate solutions are: %e and %e" % (sol1, sol2))
    
    

The accurate solutions are: -1.000000e-06 and -1.000000e+06


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare it with the answer your program gives. The two will not agree perfectly. Why?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [15]:
delta = 1e-2
x = 1

def f(x):
    return x*(x-1)

def derivative(f, x, delta):
    return (f(x+delta)-f(x))/delta

def true_deriv(x):
    return 2*x-1


print("\nThe derivative using the definition is %f" % derivative(f, x, delta))
print("\nThe true value is %f" % true_deriv(x))
per = abs((true_deriv(x)-derivative(f, x, delta))/true_deriv(x))*100
print("\nBoth values differ in %.1f per cent" % per)


The derivative using the definition is 1.010000

The true value is 1.000000

Both values differ in 1.0 per cent


The two values will not agree perfectly for the same reason we have commented in the previous exercise. In this case $f(1) = 0$ and $f(1+\delta)\approx 0$ so we have a numerator that is very close to zero, divided by a number that is also very close to zero (and therefore the unprecission in the numerator gets amplified).

In [16]:
for i in range(2, 8):
    delta = 10**(-2*i)
    print("\nThe derivative for delta = %1.0e, is: %f" % (delta,derivative(f, x, delta)))
    
    per = abs((true_deriv(x)-derivative(f, x, delta))/true_deriv(x))*100
    print("Both values differ in %.5f per cent" % per)


The derivative for delta = 1e-04, is: 1.000100
Both values differ in 0.01000 per cent

The derivative for delta = 1e-06, is: 1.000001
Both values differ in 0.00010 per cent

The derivative for delta = 1e-08, is: 1.000000
Both values differ in 0.00000 per cent

The derivative for delta = 1e-10, is: 1.000000
Both values differ in 0.00001 per cent

The derivative for delta = 1e-12, is: 1.000089
Both values differ in 0.00889 per cent

The derivative for delta = 1e-14, is: 0.999201
Both values differ in 0.07993 per cent


There seems to be a maximum precission at around $\delta = 10^{-8}$ to get lower afterwards. In fact, if the value of $\delta$ keeps dropping, the calculation will not be precise enough to compute the numerator and the result will yield a derivative equal to 0.

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which is known to be $I=\frac{\pi}{2}=1.57079632679...$.

Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a program to compute the integral with $N=100$. How does the result compare to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [17]:
import numpy as np

def f(x):
    return np.sqrt(1-x**2)

N = 100
h = 2./N
I = 0
for i in range(N+1):
    y_k = f(-1+i*h)
    I += h*y_k

print("\nThe result is I = %.11f" % I)
per = abs(I-np.pi/2)*2/np.pi*100
print("It differs from the true value by %.2f per cent." % per)


The result is I = 1.56913425555
It differs from the true value by 0.11 per cent.


#### Attempt 1
The following code is an authomatic algorithm to compute a certain number of iterations (within a given precission) in which to calculate the integral. However, in jupyter notebook it seeems to not work properly (while at my own terminal does fine). I attached it here for completeness, although I used a simpler method to solve section b) of the exercise.

In [21]:
import time
# calculate the value N for which the calculation takes a little less than 1 sec
start_time = time.time()
N = 100
while(time.time()-start_time < 1.):
    h = 2./N
    I = 0

    for i in range(N+1):
        y_k = f(-1+i*h)
        I += h*y_k

    if ((time.time()-start_time)<0.9):
        N *= 2
        start_time = time.time()
    else:
        N = int(N*(9/10))

# calculate and confirm it takes less than 1 sec
start_time = time.time()
h = 2./N
I = 0
for i in range(N+1):
    y_k = f(-1+i*h)
    I += h*y_k

print("\nTime of execution: ", time.time()-start_time)
print("The result is I = %.11f" % I)
per = abs(I-np.pi/2)*2/np.pi*100
print("It differs from the true value by %.2f per cent." % per)



Time of execution:  0.9109361171722412
The result is I = 1.57079632417
It differs from the true value by 0.00 per cent.


In [22]:
start_time = time.time()
N = 750000
h = 2./N
I = 0
for i in range(N+1):
    y_k = f(-1+i*h)
    I += h*y_k

print("\nThe result is I = %.11f" % I)
per = abs(I-np.pi/2)*2/np.pi*100
print("It differs from the true value by %.10f per cent." % per)
print("\nTime of execution: ", time.time()-start_time)



The result is I = 1.57079632423
It differs from the true value by 0.0000001630 per cent.

Time of execution:  0.9740498065948486


In [23]:
import time
start_time = time.time()
N = 50000000
h = 2./N
I = 0
for i in range(N+1):
    y_k = f(-1+i*h)
    I += h*y_k

print("\nThe result is I = %.11f" % I)
per = abs(I-np.pi/2)*2/np.pi*100
print("It differs from the true value by %.10f per cent." % per)
print("\nTime of execution: ", time.time()-start_time)


The result is I = 1.57079632679
It differs from the true value by 0.0000000003 per cent.

Time of execution:  59.05505585670471


As we can see, the difference in percentage of both calculations are negligible (around $10^{-7}$%), which does not compensate the 60 times slower algorithm.